<a href="https://colab.research.google.com/github/relssek98/MLTeam54/blob/main/Logistic_Regression_%2B_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from nltk.probability import FreqDist
nltk.download('punkt')
from sklearn import naive_bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score


#added
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification



from google.colab import drive
drive.mount('/content/drive', force_remount = True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Mounted at /content/drive


#Open Data

In [ ]:
data = pd.read_csv(r"/content/drive/MyDrive/Machine Learning/Data/SDoutput1.csv")
from ast import literal_eval
data['clean'] = data['clean'].apply(literal_eval)
data.loc[data["label"]=='suicide', "label"] = 1.0
data.loc[data["label"]=='non-suicide', "label"] = 0.0
print(type(data["label"][0]))

data.head()

<class 'float'>


,Unnamed: 0,clean,label
0,0,"[ex, wife, threaten, suiciderec, left, wife, g...",1.0
1,1,"[weird, get, affect, compliment, come, someon,...",0.0
2,2,"[final, 2020, almost, never, hear, 2020, ha, b...",0.0
3,3,"[need, helpjust, help, cri, hard]",1.0
4,4,"[losthello, name, adam, 16, struggl, year, afr...",1.0


#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
103569
  (0, 37423)	2
  (0, 91283)	1
  (0, 55814)	2
  (0, 59402)	1
  (0, 87613)	1
  (0, 68970)	1
  (0, 67253)	1
  (0, 90839)	1
  (0, 73080)	1
  (0, 84052)	1
  (0, 28500)	1
  (0, 57634)	3
  (0, 36850)	1
  (0, 28155)	1
  (0, 25617)	3
  (0, 17120)	1
  (0, 80576)	1
  (0, 28034)	1
  (0, 3261)	1
  (0, 58498)	1
  (0, 46009)	1
  (0, 96826)	1
  (0, 23385)	1
  (0, 100224)	1
  (0, 69333)	1
  :	:
  (99963, 12837)	1
  (99963, 39120)	1
  (99964, 36467)	1
  (99964, 77361)	1
  (99964, 101712)	1
  (99965, 40181)	1
  (99965, 71483)	1
  (99965, 13154)	1
  (99965, 58244)	1
  (99965, 67475)	1
  (99965, 80262)	1
  (99965, 26327)	1
  (99965, 79429)	1
  (99966, 90784)	2
  (99966, 44226)	1
  (99966, 75690)	1
  (99966, 83023)	1
  (99966, 47670)	1
  (99966, 101160)	1
  (99966, 81453)	1
  (99966, 73184)	1
  (99966, 17586)	1
  (99966, 45943)	1
  (99966, 13091)	1
  (99966, 39405)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def randomForest(x_train, y_train, x_test, y_test): 
  classif = RandomForestClassifier(n_estimators= 50)
  classif.fit(x_train, y_train)
  y_predict = classif.predict(x_test)
  print(metrics.accuracy_score(y_test, y_predict))


def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




In [ ]:
randomForest(x_train_bow, y_train, X_te_bow, y_test)

0.8895340985820391


##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
102618
  (0, 30878)	1
  (0, 13904)	1
  (0, 97957)	2
  (0, 14959)	2
  (0, 73860)	1
  (0, 70824)	1
  (0, 65344)	1
  (0, 40638)	1
  (0, 26809)	2
  (0, 83614)	1
  (0, 53741)	2
  (0, 98826)	1
  (0, 15878)	1
  (0, 20412)	1
  (1, 53741)	1
  (1, 86724)	1
  (1, 28746)	1
  (1, 24856)	1
  (1, 91242)	1
  (1, 27518)	3
  (1, 29853)	1
  (1, 66411)	1
  (1, 25660)	2
  (1, 36108)	1
  (1, 44549)	2
  :	:
  (99966, 21522)	1
  (99966, 42880)	1
  (99966, 94982)	1
  (99966, 33619)	1
  (99966, 92244)	2
  (99966, 16578)	1
  (99966, 85501)	1
  (99966, 55581)	1
  (99966, 71912)	1
  (99966, 47718)	2
  (99966, 92745)	2
  (99966, 52095)	1
  (99966, 17755)	3
  (99966, 63742)	2
  (99966, 17061)	1
  (99966, 39061)	1
  (99966, 46206)	1
  (99966, 35213)	1
  (99966, 3022)	2
  (99966, 78898)	1
  (99966, 79945)	1
  (99966, 4074)	1
  (99966, 75928)	1
  (99966, 74876)	1
  (99966, 35967)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
98698
  (0, 38100)	1
  (0, 45406)	1
  (0, 26160)	1
  (0, 79959)	1
  (1, 59414)	1
  (1, 79700)	1
  (1, 84182)	1
  (1, 87760)	1
  (1, 71137)	1
  (1, 12874)	1
  (1, 20700)	1
  (1, 73084)	1
  (1, 80169)	1
  (1, 35724)	1
  (1, 96866)	1
  (1, 68329)	1
  (1, 85078)	1
  (1, 55891)	1
  (2, 51421)	1
  (2, 51023)	1
  (2, 51846)	2
  (2, 40131)	2
  (2, 13142)	1
  (2, 38945)	1
  (2, 69107)	1
  :	:
  (99966, 60324)	1
  (99966, 94534)	1
  (99966, 55634)	1
  (99966, 35186)	1
  (99966, 74220)	1
  (99966, 59372)	1
  (99966, 69524)	2
  (99966, 93912)	1
  (99966, 14246)	1
  (99966, 96953)	1
  (99966, 70588)	3
  (99966, 45893)	1
  (99966, 19452)	1
  (99966, 66603)	1
  (99966, 69466)	1
  (99966, 39250)	1
  (99966, 84027)	1
  (99966, 93429)	1
  (99966, 73642)	1
  (99966, 39417)	1
  (99966, 42154)	1
  (99966, 27412)	1
  (99966, 34805)	1
  (99966, 47027)	1
  (99966, 71663)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
102793
  (0, 17692)	2
  (0, 39656)	3
  (0, 29749)	2
  (0, 63000)	1
  (0, 65039)	1
  (0, 84206)	1
  (0, 61126)	2
  (0, 71887)	2
  (0, 91645)	2
  (0, 42945)	3
  (0, 44154)	3
  (0, 30633)	7
  (0, 68941)	3
  (0, 36824)	3
  (0, 90101)	6
  (0, 54206)	2
  (0, 74557)	2
  (0, 80020)	3
  (0, 83017)	4
  (0, 57137)	3
  (0, 25196)	3
  (0, 58875)	1
  (0, 66847)	1
  (0, 69983)	1
  (0, 55527)	2
  :	:
  (99966, 88588)	1
  (99966, 77310)	2
  (99966, 53177)	1
  (99966, 34222)	1
  (99966, 81179)	1
  (99966, 18354)	1
  (99966, 18351)	1
  (99966, 41945)	1
  (99966, 70071)	1
  (99966, 81388)	3
  (99966, 91399)	1
  (99966, 57078)	3
  (99966, 13019)	1
  (99966, 69854)	3
  (99966, 30682)	1
  (99966, 86187)	1
  (99966, 88253)	1
  (99966, 18484)	1
  (99966, 83392)	1
  (99966, 29589)	1
  (99966, 84674)	1
  (99966, 87056)	1
  (99966, 101062)	1
  (99966, 20529)	1
  (99966, 61435)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
100067
  (0, 95281)	1
  (0, 71860)	2
  (0, 80667)	2
  (0, 92079)	1
  (0, 72004)	1
  (0, 77629)	2
  (0, 27765)	1
  (0, 34027)	1
  (0, 42521)	1
  (0, 47880)	1
  (0, 72033)	1
  (0, 30537)	1
  (0, 31375)	1
  (0, 41255)	1
  (0, 9045)	1
  (0, 27646)	1
  (0, 41375)	1
  (0, 82554)	1
  (0, 24957)	1
  (0, 38676)	1
  (0, 52008)	1
  (0, 50629)	1
  (0, 33833)	1
  (0, 92695)	1
  (1, 27692)	2
  :	:
  (99965, 54025)	1
  (99965, 54952)	1
  (99965, 98921)	1
  (99965, 39689)	1
  (99965, 21390)	1
  (99965, 15050)	1
  (99965, 67281)	1
  (99965, 91339)	1
  (99965, 89745)	1
  (99965, 34014)	1
  (99965, 28036)	1
  (99966, 77632)	1
  (99966, 52623)	1
  (99966, 11586)	1
  (99966, 38485)	1
  (99966, 9758)	1
  (99966, 96617)	2
  (99966, 66456)	1
  (99966, 81562)	1
  (99966, 94365)	1
  (99966, 58148)	1
  (99966, 22165)	1
  (99966, 29109)	1
  (99966, 12503)	3
  (99966, 1351)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
102339
  (0, 64201)	1
  (0, 69061)	1
  (0, 34733)	1
  (0, 40821)	1
  (0, 86431)	1
  (0, 41384)	1
  (0, 74620)	1
  (0, 78003)	1
  (0, 28435)	1
  (1, 40821)	1
  (1, 100470)	2
  (1, 50264)	3
  (1, 12213)	1
  (1, 53018)	1
  (1, 87002)	2
  (1, 42628)	2
  (1, 77770)	1
  (1, 67002)	1
  (1, 207)	1
  (1, 100623)	1
  (1, 38613)	1
  (1, 49687)	1
  (1, 97654)	1
  (1, 79503)	1
  (1, 12888)	1
  :	:
  (99964, 21391)	1
  (99965, 69061)	1
  (99965, 55217)	1
  (99965, 100668)	1
  (99965, 66215)	1
  (99965, 69365)	1
  (99965, 80221)	1
  (99965, 51671)	1
  (99965, 84949)	1
  (99965, 99912)	1
  (99966, 66039)	1
  (99966, 89542)	2
  (99966, 41058)	1
  (99966, 95900)	1
  (99966, 62789)	1
  (99966, 47232)	1
  (99966, 27058)	1
  (99966, 55160)	1
  (99966, 14824)	1
  (99966, 19704)	1
  (99966, 60480)	2
  (99966, 14382)	1
  (99966, 33130)	1
  (99966, 50018)	1
  (99966, 83657)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
101482
  (0, 74366)	1
  (0, 82756)	1
  (0, 41274)	1
  (0, 16006)	1
  (0, 78296)	1
  (0, 68889)	1
  (0, 27125)	1
  (0, 2926)	1
  (0, 20205)	1
  (0, 37253)	1
  (0, 37264)	16
  (1, 23007)	1
  (1, 25273)	1
  (1, 86987)	1
  (1, 68193)	1
  (2, 87311)	1
  (2, 26719)	1
  (2, 69915)	1
  (2, 12552)	1
  (2, 95116)	1
  (2, 42970)	1
  (2, 55118)	1
  (2, 13702)	1
  (3, 42970)	1
  (3, 68596)	1
  :	:
  (99965, 26500)	1
  (99965, 35895)	1
  (99965, 36600)	1
  (99965, 54485)	1
  (99965, 79262)	1
  (99965, 72248)	1
  (99965, 19895)	1
  (99965, 90787)	1
  (99965, 16879)	1
  (99965, 83627)	1
  (99965, 10793)	1
  (99965, 79644)	1
  (99965, 94160)	1
  (99965, 74826)	1
  (99965, 20123)	1
  (99965, 92531)	1
  (99965, 49573)	1
  (99965, 19064)	1
  (99965, 89052)	1
  (99966, 41921)	1
  (99966, 70817)	1
  (99966, 75054)	1
  (99966, 15646)	1
  (99966, 33203)	1
  (99966, 82022)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
96699
  (0, 7145)	1
  (0, 65348)	4
  (0, 69847)	1
  (0, 79030)	1
  (0, 37367)	1
  (0, 90472)	1
  (0, 15794)	1
  (0, 83742)	1
  (1, 73742)	2
  (1, 64302)	2
  (1, 39500)	1
  (1, 87216)	1
  (1, 38715)	1
  (1, 34030)	1
  (1, 56550)	1
  (1, 7684)	1
  (1, 13725)	1
  (1, 70595)	2
  (1, 70524)	1
  (1, 42105)	4
  (1, 6810)	1
  (1, 43901)	2
  (1, 29174)	1
  (1, 11282)	1
  (1, 44624)	1
  :	:
  (99966, 76393)	1
  (99966, 85472)	1
  (99966, 67118)	1
  (99966, 71047)	3
  (99966, 67390)	1
  (99966, 54139)	1
  (99966, 55140)	2
  (99966, 13598)	2
  (99966, 2155)	1
  (99966, 63131)	1
  (99966, 54623)	1
  (99966, 45113)	5
  (99966, 10277)	1
  (99966, 46857)	1
  (99966, 56994)	14
  (99966, 54522)	1
  (99966, 86334)	1
  (99966, 45114)	1
  (99966, 84453)	1
  (99966, 86491)	1
  (99966, 52020)	1
  (99966, 13390)	1
  (99966, 60317)	1
  (99966, 64065)	1
  (99966, 59069)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
101855
  (0, 36307)	3
  (0, 31725)	1
  (0, 57226)	1
  (0, 30008)	1
  (0, 56282)	1
  (0, 10606)	1
  (0, 88897)	1
  (0, 89188)	1
  (0, 77112)	1
  (0, 99966)	3
  (0, 17581)	1
  (0, 85537)	1
  (0, 73702)	1
  (0, 71843)	1
  (0, 62653)	2
  (0, 16061)	1
  (0, 29204)	1
  (0, 13336)	2
  (0, 30211)	1
  (0, 10455)	1
  (0, 49007)	1
  (0, 88378)	1
  (0, 81849)	4
  (0, 68935)	1
  (0, 90140)	3
  :	:
  (99966, 90140)	1
  (99966, 45272)	1
  (99966, 13669)	2
  (99966, 55184)	3
  (99966, 33228)	2
  (99966, 12053)	1
  (99966, 89039)	1
  (99966, 79121)	1
  (99966, 30660)	3
  (99966, 82850)	1
  (99966, 39152)	1
  (99966, 54348)	1
  (99966, 97483)	1
  (99966, 57615)	1
  (99966, 36871)	3
  (99966, 68743)	1
  (99966, 56499)	2
  (99966, 17221)	1
  (99966, 36139)	1
  (99966, 11087)	1
  (99966, 50371)	1
  (99966, 101313)	1
  (99966, 31734)	1
  (99966, 46535)	1
  (99966, 10095)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
101201
  (0, 100547)	1
  (0, 43218)	1
  (0, 96862)	1
  (0, 83444)	1
  (0, 61000)	2
  (0, 91468)	1
  (0, 41372)	1
  (0, 81439)	1
  (0, 77785)	1
  (0, 44010)	1
  (0, 88599)	1
  (0, 96375)	1
  (0, 73944)	1
  (0, 41738)	1
  (0, 26906)	1
  (0, 78049)	1
  (1, 41372)	1
  (1, 73944)	1
  (1, 92724)	1
  (1, 9804)	1
  (1, 78678)	1
  (1, 41013)	1
  (1, 2914)	2
  (1, 97230)	2
  (1, 26774)	1
  :	:
  (99965, 75090)	1
  (99965, 86425)	1
  (99965, 36436)	1
  (99965, 48069)	1
  (99965, 73647)	2
  (99966, 43218)	1
  (99966, 91468)	1
  (99966, 41372)	1
  (99966, 52976)	1
  (99966, 78138)	1
  (99966, 65503)	1
  (99966, 22050)	1
  (99966, 89589)	1
  (99966, 25310)	1
  (99966, 16066)	1
  (99966, 47163)	1
  (99966, 92065)	2
  (99966, 25393)	1
  (99966, 31864)	1
  (99966, 14952)	1
  (99966, 32261)	1
  (99966, 89786)	1
  (99966, 59299)	1
  (99966, 1079)	1
  (99966, 1318)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
99362
  (0, 34598)	1
  (0, 38316)	1
  (0, 46847)	1
  (0, 82628)	1
  (0, 56196)	1
  (0, 42006)	1
  (0, 78644)	2
  (0, 94385)	9
  (0, 1330)	1
  (0, 98243)	1
  (0, 62924)	1
  (0, 72832)	1
  (0, 56131)	1
  (0, 6653)	2
  (0, 16662)	1
  (0, 17775)	1
  (0, 52272)	1
  (0, 84724)	3
  (0, 57746)	1
  (0, 88381)	1
  (0, 23815)	1
  (0, 86446)	1
  (0, 61533)	1
  (0, 81159)	1
  (0, 63087)	2
  :	:
  (99965, 9723)	1
  (99965, 20871)	1
  (99965, 70493)	1
  (99965, 38572)	1
  (99966, 38499)	1
  (99966, 39151)	1
  (99966, 84652)	1
  (99966, 86479)	1
  (99966, 77807)	1
  (99966, 36627)	1
  (99966, 50050)	1
  (99966, 71697)	1
  (99966, 68405)	1
  (99966, 69192)	1
  (99966, 10263)	1
  (99966, 92865)	1
  (99966, 53713)	1
  (99966, 93630)	1
  (99966, 61034)	1
  (99966, 61836)	1
  (99966, 15812)	1
  (99966, 13203)	1
  (99966, 74014)	1
  (99966, 36101)	1
  (99966, 13274)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
102723
  (0, 89989)	1
  (0, 41770)	2
  (0, 54389)	1
  (0, 98691)	1
  (0, 91178)	1
  (0, 19232)	1
  (0, 92839)	1
  (0, 13590)	1
  (1, 41770)	4
  (1, 91178)	1
  (1, 92839)	1
  (1, 37054)	1
  (1, 81886)	1
  (1, 12144)	2
  (1, 35242)	1
  (1, 81887)	1
  (1, 67738)	1
  (1, 37139)	5
  (1, 31828)	1
  (1, 38087)	1
  (1, 78323)	1
  (1, 36211)	1
  (1, 66628)	2
  (1, 69195)	2
  (1, 83651)	4
  :	:
  (99966, 19597)	2
  (99966, 46831)	1
  (99966, 82689)	1
  (99966, 88380)	1
  (99966, 65147)	1
  (99966, 79159)	1
  (99966, 58985)	1
  (99966, 90837)	1
  (99966, 25173)	1
  (99966, 102316)	1
  (99966, 58503)	1
  (99966, 73786)	1
  (99966, 64331)	1
  (99966, 72772)	1
  (99966, 43932)	1
  (99966, 91892)	1
  (99966, 22136)	1
  (99966, 46819)	1
  (99966, 48258)	1
  (99966, 65699)	1
  (99966, 24554)	1
  (99966, 23818)	1
  (99966, 70588)	1
  (99966, 59736)	1
  (99966, 93335)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
98044
  (0, 74507)	1
  (0, 85411)	1
  (0, 86489)	1
  (0, 7837)	1
  (0, 93490)	2
  (0, 37479)	1
  (0, 39934)	3
  (0, 62706)	2
  (0, 93234)	3
  (0, 28635)	1
  (0, 62270)	1
  (0, 74527)	3
  (0, 67967)	2
  (0, 23881)	1
  (0, 69913)	2
  (0, 51609)	1
  (0, 11985)	1
  (0, 18519)	1
  (0, 69877)	1
  (0, 13575)	1
  (0, 38059)	1
  (0, 12254)	1
  (0, 70736)	1
  (0, 49691)	1
  (0, 65255)	1
  :	:
  (99965, 56529)	1
  (99965, 27276)	1
  (99965, 79621)	1
  (99965, 30294)	1
  (99965, 94742)	1
  (99965, 83011)	1
  (99965, 87234)	1
  (99965, 77680)	1
  (99965, 31740)	2
  (99965, 6703)	1
  (99965, 424)	1
  (99965, 79365)	1
  (99965, 76561)	1
  (99965, 96979)	2
  (99965, 73337)	1
  (99965, 79992)	1
  (99966, 23881)	1
  (99966, 51609)	1
  (99966, 38059)	1
  (99966, 87319)	1
  (99966, 12627)	1
  (99966, 20525)	1
  (99966, 15323)	1
  (99966, 22779)	1
  (99966, 92191)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
98851
  (0, 17163)	1
  (0, 31480)	1
  (0, 62611)	1
  (0, 44015)	1
  (0, 71498)	1
  (0, 86461)	1
  (0, 96984)	1
  (0, 34190)	1
  (0, 65600)	1
  (0, 17809)	1
  (0, 37997)	1
  (0, 40599)	1
  (0, 53329)	1
  (0, 39964)	2
  (0, 87013)	1
  (0, 10532)	1
  (0, 35666)	1
  (0, 32825)	1
  (0, 43617)	1
  (0, 94092)	1
  (0, 36810)	1
  (0, 4265)	1
  (0, 97740)	1
  (0, 23214)	1
  (0, 82656)	1
  :	:
  (99966, 79704)	1
  (99966, 94706)	1
  (99966, 14990)	2
  (99966, 81447)	1
  (99966, 83265)	1
  (99966, 87584)	1
  (99966, 89494)	1
  (99966, 17377)	1
  (99966, 24766)	1
  (99966, 82197)	1
  (99966, 96672)	2
  (99966, 62308)	1
  (99966, 57848)	1
  (99966, 80493)	1
  (99966, 43495)	1
  (99966, 58544)	1
  (99966, 20563)	1
  (99966, 43465)	1
  (99966, 78232)	1
  (99966, 40881)	2
  (99966, 78879)	1
  (99966, 89670)	1
  (99966, 50213)	1
  (99966, 80984)	1
  (99966, 55147)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
98625
  (0, 93885)	1
  (0, 43135)	1
  (0, 41627)	1
  (0, 79573)	1
  (0, 22778)	1
  (0, 59598)	1
  (0, 6293)	1
  (0, 29546)	1
  (0, 60872)	2
  (0, 37284)	2
  (0, 14200)	2
  (0, 6442)	1
  (1, 50902)	1
  (1, 15027)	1
  (1, 87225)	1
  (1, 75808)	1
  (1, 77197)	1
  (1, 85845)	1
  (1, 96446)	1
  (1, 39957)	1
  (1, 13587)	1
  (2, 85996)	1
  (2, 20774)	1
  (2, 71065)	1
  (2, 52121)	1
  :	:
  (99966, 58047)	2
  (99966, 20506)	1
  (99966, 310)	2
  (99966, 38268)	4
  (99966, 62964)	1
  (99966, 54911)	1
  (99966, 77552)	1
  (99966, 38880)	1
  (99966, 22034)	1
  (99966, 22953)	1
  (99966, 29152)	1
  (99966, 96897)	1
  (99966, 76223)	1
  (99966, 68985)	1
  (99966, 29153)	10
  (99966, 67504)	2
  (99966, 7343)	1
  (99966, 30594)	1
  (99966, 88460)	3
  (99966, 77586)	1
  (99966, 64380)	8
  (99966, 30938)	1
  (99966, 64381)	7
  (99966, 23062)	2
  (99966, 47447)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)

#Mult *Run*

##Split for Train

In [ ]:
training_data, test_data = train_test_split(data, train_size = 0.6)
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,1], lowercase=False) 
#Adjust for ngrams later
print(training_data.shape)
print(test_data.shape)

x_train_bow = bow_transform.fit_transform(training_data['clean'])
print(len(bow_transform.vocabulary_))
print(x_train_bow)

X_te_bow = bow_transform.transform(test_data['clean'])

y_train = training_data['label']
y_train=y_train.astype('int')
y_test = test_data['label']
y_test=y_test.astype('int')


(99967, 3)
(66645, 3)
102935
  (0, 74892)	2
  (0, 32452)	2
  (0, 47173)	2
  (0, 68702)	2
  (0, 57064)	1
  (0, 83966)	3
  (0, 87222)	1
  (0, 57087)	2
  (0, 86965)	2
  (0, 66480)	1
  (0, 28048)	1
  (0, 39468)	1
  (0, 46599)	1
  (0, 79384)	1
  (0, 32712)	1
  (0, 90105)	3
  (0, 84892)	1
  (0, 31959)	1
  (0, 14008)	1
  (0, 101787)	2
  (0, 80553)	1
  (0, 21380)	2
  (0, 66437)	1
  (0, 45398)	1
  (0, 78416)	1
  :	:
  (99965, 23080)	1
  (99965, 61226)	1
  (99965, 45082)	1
  (99965, 67121)	1
  (99965, 25018)	1
  (99965, 20405)	2
  (99965, 54311)	2
  (99965, 96211)	1
  (99965, 76619)	1
  (99965, 87687)	1
  (99965, 66079)	1
  (99965, 63958)	1
  (99965, 83282)	1
  (99965, 16758)	1
  (99965, 48421)	1
  (99965, 11047)	1
  (99965, 31431)	1
  (99965, 40355)	1
  (99965, 85624)	1
  (99965, 100598)	1
  (99965, 46741)	1
  (99965, 44546)	1
  (99966, 36885)	1
  (99966, 18962)	1
  (99966, 94201)	1


##TF-IDF

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(x_train_bow)
# print(X_tr_tfidf)
X_te_tfidf = tfidf_transform.transform(X_te_bow)


##Logistic Regression

###Functions

In [ ]:
def logRegFunction(x_train, y_train, x_test, y_test, info):
  logReg = LogisticRegression(max_iter=100)
  logReg.fit(x_train, y_train)
  y_predicted_prob = logReg.predict_proba(x_test)
  y_predicted = logReg.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def mnNaiveBayesFunction(x_train, y_train, x_test, y_test, info):
  classifier = naive_bayes.MultinomialNB()
  classifier.fit(x_train, y_train)
  y_predicted = classifier.predict(x_test)
  return (info,) + metricsCalc(y_test, y_predicted)

def metricsCalc(y_test, y_predicted):
  #acc score
  accuracy = accuracy_score(y_test.tolist(), y_predicted)

  #create seperate dataframes
  accuracyData = pd.DataFrame(
      {'Y' : y_test.tolist(), 'Y Predicted': y_predicted.tolist()} )
  GroundTruthSuicidal = accuracyData[accuracyData["Y"] == 1]
  GroundTruthNonSuicidal = accuracyData[accuracyData["Y"] == 0]

  #True Positive and False Negative Counts
  true_positive_count = GroundTruthSuicidal['Y Predicted'].tolist().count(1)
  false_negative_count = GroundTruthSuicidal['Y Predicted'].tolist().count(0)

  #True Negative and False Positive Counts
  true_negative_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(0)
  false_positive_count = GroundTruthNonSuicidal['Y Predicted'].tolist().count(1)

  #True Positive, True Negative, Precision 
  true_positive_rate = accuracy_score(GroundTruthSuicidal['Y'], GroundTruthSuicidal['Y Predicted'])
  true_negative_rate = accuracy_score(GroundTruthNonSuicidal['Y'], GroundTruthNonSuicidal['Y Predicted'])
  positive_precision = true_positive_count/(true_positive_count + false_positive_count)
  f1 = f1_score(y_test.tolist(), y_predicted.tolist())
  return accuracy, true_positive_rate, true_negative_rate, positive_precision, f1

def addDataToFile(results):
  file1 = open("/content/drive/MyDrive/Machine Learning/Data/results.csv", "a+")
  file1.write((','.join(str(item) for item in results)) + "\n")
  file1.close()




##Bag of Words Logistic Regression

In [ ]:
results_bow_log = logRegFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Logistic Regression")
addDataToFile(results_bow_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##TDIDF Log Reg

In [ ]:
results_TDIF_log = logRegFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TDIDF Logistic Regression")
addDataToFile(results_TDIF_log)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Bag of Words Multinomial Naive Bayes

In [ ]:
results_BOW_mnb = mnNaiveBayesFunction(x_train_bow, y_train, X_te_bow, y_test, "Bag of Words Multinomial Naive Bayes")
addDataToFile(results_BOW_mnb)

##TFIDF Multinomial Naive Bayes

In [ ]:
results_TDIF_mnb = mnNaiveBayesFunction(X_tr_tfidf, y_train, X_te_tfidf, y_test, "TFIDF Multinomial Naive Bayes")
addDataToFile(results_TDIF_mnb)